### YOLOv8 demo for CAST

<b>Run block for required dependencies:</b>

In [ ]:
# use this block to download requirements into virtual environment
%pip install -r requirements.txt

<u>Import statements<u>

In [3]:
import os
from ultralytics import YOLO
import cv2
import torch
import requests
import zipfile
import shutil

<u>Importing training dataset for demo</u>

For this demo, we will be training a YOLOv8 on African wildlife imagery pulled from https://ultralytics.com/assets/african-wildlife.zip. From ultralytics, "This dataset showcases four common animal classes typically found in South African nature reserves. It includes images of African wildlife such as buffalo, elephant, rhino, and zebra, providing valuable insights into their characteristics. Essential for training computer vision algorithms, this dataset aids in identifying animals in various habitats, from zoos to forests, and supports wildlife research.... 

The African wildlife objects detection dataset is split into three subsets:

Training set: Contains 1052 images, each with corresponding annotations.
Validation set: Includes 225 images, each with paired annotations.
Testing set: Comprises 227 images, each with paired annotations."

In [14]:
# using requests, download zip file and extract contents

# url of zipfile
url_of_zip = 'https://ultralytics.com/assets/african-wildlife.zip'
url_of_yaml = 'https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/datasets/african-wildlife.yaml'

# directory to be saved into
wd = os.getcwd()
zip_directory = os.path.join(wd, 'animals.zip')
yaml_directory = os.path.join(wd, 'data.yaml')

# download request
response = requests.get(url_of_zip)
with open(zip_directory, 'wb') as f:
    f.write(response.content)

response2 = requests.get(url_of_yaml)
with open(yaml_directory, 'wb') as f:
    f.write(response2.content)

# use the working directory as the extraction target
extract_to = wd  # Since we're extracting to the working directory

# extract the zip file
with zipfile.ZipFile(zip_directory, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# list extracted files
print(f'Extracted files: {os.listdir(extract_to)}' )


Extracted files: ['valid', 'video_test.py', 'unneeded_files', '.DS_Store', 'requirements.txt', 'test', 'experiments', 'hpc_other', 'YOLO_demo.ipynb', 'car_detector.ipynb', 'README.md', 'image-toolkit.py', 'data.yaml', '.gitattributes', 'train', 'slurm.bash', 'animals.zip', '.git', 'upload']


<b>Clear directory of training files if needed:</b>

In [4]:
# initialize file name
file_name = 'animals.zip'
file_name2 = 'data.yaml'

# construct the file path
file_path = os.path.join(os.getcwd(), file_name)
file_path2 = os.path.join(os.getcwd(), file_name2)

# delete the zip file
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"The file {file_name} has been deleted.")
else:
    print(f"The file {file_name} does not exist.")

# delete the zip file
if os.path.exists(file_path2):
    os.remove(file_path2)
    print(f"The file {file_name2} has been deleted.")
else:
    print(f"The file {file_name2} does not exist.")

# list of directory paths you want to delete
directories = ["./train", "./test", "./valid"]

# loop through each directory in the list
for directory_path in directories:

    # check if the directory exists
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        
        # use shutil.rmtree() to delete the directory
        shutil.rmtree(directory_path)
        print(f"The directory {directory_path} has been deleted.")

    else:
        print(f"The directory {directory_path} does not exist.")


The file animals.zip has been deleted.
The file data.yaml has been deleted.
The directory ./train has been deleted.
The directory ./test has been deleted.
The directory ./valid has been deleted.


<u>Training the detection model on local machine</u>

This block will train the YOLO model using your local machine. Depending on the size of the training/val data and model size, this could be too computationally intensive for your hardware; we will discuss using UARK's high performance computing center if this is the case.

YOLOv8 has a handful of customizable hyperparameters you can read about here: https://docs.ultralytics.com/usage/cfg/#train-settings

This code block will prompt you for a few commonly customized hyperparameters.

<b>A note about batch size and optimizer:</b> using batch size = -1 will find the computationally optimal batch size for your local machine. Similarly, using optimizer = auto will do the same.

In [4]:
# prompt for hyperparameters
print("\033[1m" + "Hyperparameter intialization" + "\033[0m")

# epochs
print(f'Enter the number of epochs: ')
epochs = input()
print(epochs)

# batch size
print(f'Enter the batch size: ')
batch_size = input()
print(batch_size)

# optimizer
print(f'Enter the optimizer (SGD, Adam, AdamW, NAdam, RAdam, RMSProp, or auto)')
optimizer = input()
print()

Hyperparameter intialization
Enter the number of epochs: 
100


In [ ]:
# check if cuda GPU training is available. if not, set to CPU training.
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

# load a model
model = YOLO('yolov8n.pt')
model.to(device_name)

# prompt for hyperparameters
print(f'Enter desired hyperparameters: ')
print(f'Epochs: ')
epochs = input()

# train model
results = model.train(data="data.yaml", epochs=80, batch=-1, optimizer='AdamW', device=device_name, cos_lr = True)

<b>Clear runs and trained model if needed:</b>

In [ ]:
# initialize file name
file_name = 'yolov8n.pt'

# construct the file path
file_path = os.path.join(os.getcwd(), file_name)

# delete the zip file
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"The file {file_name} has been deleted.")
else:
    print(f"The file {file_name} does not exist.")

# list of directory paths you want to delete
directory_path = "./runs"

# check if the directory exists
if os.path.exists(directory_path) and os.path.isdir(directory_path):
    
    # use shutil.rmtree() to delete the directory
    shutil.rmtree(directory_path)
    print(f"The directory {directory_path} has been deleted.")

else:
    print(f"The directory {directory_path} does not exist.")
